In [1]:
import re
import json
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import pandas as pd
from tqdm import tqdm
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/phi-4-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

tokenizer = get_chat_template(tokenizer, chat_template="phi-4")
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.1: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.988 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\unsloth\models\llama.py:1287: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(100352, 5120, padding_idx=100351)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=1280, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=1280, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=17920, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=17920, bias=False)
          (down_proj): Linear4bit(in_features=17920, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((512

In [3]:
def extract_assistant_response(response):
    # Regular expression pattern to extract the assistant's reply
    pattern = r'<\|im_start\|>assistant<\|im_sep\|>(.*?)<\|im_end\|>'

    # Search for the pattern in the conversation
    match = re.search(pattern, response, re.DOTALL)

    if match:
        response = match.group(1).strip()
    return response

In [4]:
system_message = """You are an advanced AI language model with specialized expertise in accounting classifications. Your task is to determine whether a provided "item_name" belongs to the specified "account_name". For each input, generate a JSON object containing the following keys:

- "item_name": The original item name provided by the user.
- "account_name": The account name provided by the user.
- "belongs_to_account": A boolean value (true or false) indicating whether the item belongs to the provided account name.
- "confidence_status": Your confidence level in the evaluation; set this to "Sure" if your confidence is above 95% and "Not Sure" if it is below 95%.

Instructions:
1. Input: Two strings, "item_name" and "account_name".
2. Output: A JSON object strictly following this structure:
   {
     "belongs_to_account": <true or false>,
     "confidence_status": "<Sure or Not Sure>"
   }
3. Your output must be in valid JSON format with no additional text, explanations, or commentary.
4. If you are uncertain about the evaluation for any field, assign the value "Unknown" to that field.
5. Use your comprehensive accounting knowledge to ensure that your evaluations and confidence assessments are as accurate and professional as possible.
"""
# system_message = json.dumps(system_message, ensure_ascii=False)

# item_name = """\"10.000 Stck. DIN A4 Briefpapier, 90g/m² matt, 4/4-farbig, "Seite 1"\""""
# account_name = """\"Bürobedarf\""""
#
# user_message = f"Is the item_name: {item_name} belongs to the account_name: {account_name}?"
#
# messages = [
#     {"role": "system", "content": system_message},
#     {"role": "user", "content": user_message}
# ]

In [5]:
# inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(
#     "cuda")

In [6]:
# outputs = model.generate(input_ids=inputs, max_new_tokens=200, use_cache=True, temperature=0.1, min_p=0.1)

In [7]:
# output = tokenizer.batch_decode(outputs)
# assistant_response = extract_assistant_response(output[0])
# print(assistant_response)

In [8]:
# cleaned_string = re.sub(r'^```json\s*|```\s*$', '', assistant_response, flags=re.MULTILINE).strip()
# data = json.loads(cleaned_string)
# data

In [9]:
df_global_account = pd.read_excel("../dataset/total_auto_mapping_only_pararent_account.xlsx")
account_name_de = df_global_account["account_name_de"]

In [10]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, text

db_username = 'admin'
db_password = 'Q8YSDeMLsp1l6oy'
db_hostname = 'srv-caito-prod01.cisbox.com'


db_port = 3310
db_schema = 'item_allocation_cisbox'
DATABASE_URL = f"postgresql+psycopg2://{db_username}:{db_password}@{db_hostname}:{db_port}/{db_schema}"

In [11]:
engine = create_engine(DATABASE_URL, echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
# sql_query = text(
#     """
#     select * from company_accounting_item where account_name = 'Bürobedarf' order by item_name
#     """
# )
# result = session.execute(sql_query)
# rows = result.fetchall()

In [13]:
# # convert to dict
# results = [dict(row) for row in rows]
# # Convert to dataframe
# df = pd.DataFrame(results)
# df.head(10)

In [14]:
output_path = "../dataset/output"

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_directory(output_path)

for account_name in account_name_de:
    # sql_query = text(f"""select * from company_accounting_item where account_name like '%{account_name}%' order by item_name""")
    sql_query = text(f"""select * from company_accounting_item where account_name  = '{account_name}' order by item_name""")
    rows = session.execute(sql_query).fetchall()
    results = [dict(row) for row in rows]
    df_item = pd.DataFrame(results)

    for index, row in tqdm(df_item.iterrows(), total=df_item.shape[0]):
        item_name = row['item_name']
        user_message = f"Is the item_name: {item_name} belongs to the account_name: {account_name}?"
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
        inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids=inputs, max_new_tokens=200, use_cache=True, temperature=0.1, min_p=0.1)
        output = tokenizer.batch_decode(outputs)
        assistant_response = extract_assistant_response(output[0])
        cleaned_string = re.sub(r'^```json\s*|```\s*$', '', assistant_response, flags=re.MULTILINE).strip()
        data = json.loads(cleaned_string)
        try:
            belongs_to_account = data["belongs_to_account"]
            confidence_status = data["confidence_status"]
        except:
            belongs_to_account = "Unknown"
            confidence_status = "Unknown"
        df_item.at[index, "belongs_to_account"] = belongs_to_account
        df_item.at[index, "confidence_status"] = confidence_status
        # Save to file
        file_path = os.path.join(output_path, f"{account_name}.xlsx")
        df_item.to_excel(file_path, index=False)

2025-03-04 11:12:40,072 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-04 11:12:40,075 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-04 11:12:40,511 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-04 11:12:40,513 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-04 11:12:40,945 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-04 11:12:40,946 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-04 11:12:41,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-04 11:12:41,381 INFO sqlalchemy.engine.Engine select * from company_accounting_item where account_name  = 'Bürobedarf' order by item_name
2025-03-04 11:12:41,382 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {}


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21116\1806966146.py:13: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  results = [dict(row) for row in rows]
  1%|          | 602/51074 [1:57:45<164:33:03, 11.74s/it]

KeyboardInterrupt



In [ ]:
session.commit()
session.close()